# Set up

Import all packages

In [ ]:
!pip install nltk

In [ ]:
import json
import re
from string import punctuation
from nltk import word_tokenize 
from nltk.util import ngrams
import nltk
nltk.download('punkt')
from nltk.util import pad_sequence
import math
import numpy as np
import pandas as pd
from keras.preprocessing.text import text_to_word_sequence
from gensim.models import Word2Vec
import torch
from textblob import TextBlob
from tqdm.notebook import tqdm
import pickle
from scipy import spatial

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en')
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
import ast 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

Set up Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
drive_dir = '/content/gdrive/Shareddrives/CIS530Project/milestone3/'

# Preprocess

## Load dataset

Download json dataset

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2021-05-06 04:45:38--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       100%[===================>]   4.17M  26.5MB/s    in 0.2s    

2021-05-06 04:45:39 (26.5 MB/s) - ‘dev-v2.0.json’ saved [4370528/4370528]



In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2021-05-06 04:45:41--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  73.5MB/s    in 0.5s    

2021-05-06 04:45:42 (73.5 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



Convert to DF format

In [ ]:
def load_data(train_df):
    contexts = []
    questions = []
    answers_text = []
    label = []
    for i in range(train_df['data'].shape[0]):
        topic = train_df['data'].iloc[i]['paragraphs']
        for sub_para in topic:
            for q_a in sub_para['qas']:
                if(not(q_a['is_impossible'])):
                    questions.append(q_a['question'])
                    answer = q_a['answers'][0]['text']
                    answers_text.append(answer)
                    ans_start = q_a['answers'][0]['answer_start']
                    ans_end = ans_start + len(answer)
                    label.append([ans_start, ans_end])
                    contexts.append(sub_para['context'])
    df = pd.DataFrame({"contexts":contexts, "question": questions, "label": label, "text": answers_text})
    return df

In [ ]:
train_json = pd.read_json('/content/train-v2.0.json')
dev_json = pd.read_json('/content/dev-v2.0.json')
train_df = load_data(train_json)
dev_df = load_data(dev_json)

Inspect dataset

In [ ]:
train_df

contexts  ...                         text
0      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            in the late 1990s
1      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...          singing and dancing
2      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                         2003
3      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...               Houston, Texas
4      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                   late 1990s
...                                                  ...  ...                          ...
86816  Kathmandu Metropolitan City (KMC), in order to...  ...                       Oregon
86817  Kathmandu Metropolitan City (KMC), in order to...  ...                      Rangoon
86818  Kathmandu Metropolitan City (KMC), in order to...  ...                        Minsk
86819  Kathmandu Metropolitan City (KMC), in order to...  ...                         1975
86820  Kathmandu Metropolitan City (KMC), in order to...  ...  Kathmandu Metropolitan City

[86821 rows x 4 columns]

In [ ]:
paras = list(train_df["contexts"].drop_duplicates().reset_index(drop= True))
len(paras)

18877

In [ ]:
len(train_df)

86821

## Create embedding

Find all sentences

In [ ]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [ ]:
len(sentences)

92597

Import glove embedding

In [ ]:
!wget https://apache-mxnet.s3.cn-north-1.amazonaws.com.cn/gluon/embeddings/glove/glove.840B.300d.zip

--2021-04-23 05:45:21--  https://apache-mxnet.s3.cn-north-1.amazonaws.com.cn/gluon/embeddings/glove/glove.840B.300d.zip
Resolving apache-mxnet.s3.cn-north-1.amazonaws.com.cn (apache-mxnet.s3.cn-north-1.amazonaws.com.cn)... 54.222.48.81
Connecting to apache-mxnet.s3.cn-north-1.amazonaws.com.cn (apache-mxnet.s3.cn-north-1.amazonaws.com.cn)|54.222.48.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  14.4MB/s    in 2m 48s  

2021-04-23 05:48:10 (12.4 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [ ]:
!unzip /content/glove.840B.300d.zip

Archive:  /content/glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


Import infersent embedding

In [ ]:
!curl -Lo infersent1.pkl https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
!curl -Lo infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  32.9M      0  0:00:04  0:00:04 --:--:-- 32.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  36.1M      0  0:00:04  0:00:04 --:--:-- 36.5M


In [ ]:
from models import InferSent
V = 2
MODEL_PATH = 'infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
W2V_PATH = '/content/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)

In [ ]:
infersent.build_vocab(sentences, tokenize=True)

Found 88962(/109682) words with w2v vectors
Vocab size : 88962


Generate embedding

In [ ]:
dict_embeddings = {}
for i in tqdm(range(len(sentences))):
    dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)

In [ ]:
questions = list(train_df["question"])

In [ ]:
for i in tqdm(range(len(questions))):
    dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)

In [ ]:
dict_emb = pd.read_pickle(drive_dir +"dict_embeddings_combined.pkl")

In [ ]:
def get_target(x):
    idx = -1
    for i in range(len(x["sentences"])):
        if x["text"] in x["sentences"][i]: idx = i
    return idx

In [ ]:
train_df

contexts  ...                         text
0      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            in the late 1990s
1      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...          singing and dancing
2      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                         2003
3      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...               Houston, Texas
4      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...                   late 1990s
...                                                  ...  ...                          ...
86816  Kathmandu Metropolitan City (KMC), in order to...  ...                       Oregon
86817  Kathmandu Metropolitan City (KMC), in order to...  ...                      Rangoon
86818  Kathmandu Metropolitan City (KMC), in order to...  ...                        Minsk
86819  Kathmandu Metropolitan City (KMC), in order to...  ...                         1975
86820  Kathmandu Metropolitan City (KMC), in order to...  ...  Kathmandu Metropolitan City

[86821 rows x 4 columns]

In [ ]:
def process_data(train):
    train['sentences'] = train['contexts'].apply(lambda x: [item.raw for item in TextBlob(x).sentences]) 
    train["target"] = train.apply(get_target, axis = 1)
    train['sent_emb'] = train['sentences'].apply(lambda x: [dict_emb[item][0] if item in\
                                                           dict_emb else np.zeros(4096) for item in x])
    train['quest_emb'] = train['question'].apply(lambda x: dict_emb[x] if x in dict_emb else np.zeros(4096) )
        
    return train

In [ ]:
train_df = process_data(train_df)

In [ ]:
train_df

contexts  ...                                          quest_emb
0      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...  [[-0.0287438, 0.12983947, -0.07295281, 0.02881...
1      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...  [[0.04561126, 0.1544995, 0.048832703, 0.067307...
2      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...  [[-0.028743807, 0.12983945, 0.023517007, -0.00...
3      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...  [[0.006076671, 0.1262721, 0.03748754, 0.031985...
4      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...  [[0.0037452257, 0.10884415, 0.0060910997, 0.06...
...                                                  ...  ...                                                ...
86816  Kathmandu Metropolitan City (KMC), in order to...  ...  [[0.015823055, 0.04943118, 0.047824867, 0.0656...
86817  Kathmandu Metropolitan City (KMC), in order to...  ...  [[0.045611262, 0.113482654, 0.07709428, 0.0312...
86818  Kathmandu Metropolitan City (KMC), in order to...  ...  [[0.062544174, 0.0983075, 0.081294075, 0.04585...
86819  Kathmandu Metropolitan City (KMC), in order to...  ...  [[0.0060766675, 0.04943118, 0.05144073, 0.1033...
86820  Kathmandu Metropolitan City (KMC), in order to...  ...  [[0.045611262, 0.04943118, 0.042087067, 0.0312...

[86821 rows x 8 columns]

## Generate features

### Cosine similarity

Compute cosine similarity between context sentences and questions

In [ ]:
def cosine_sim(x):
    li = []
    for item in x["sent_emb"]:
        li.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return li

In [ ]:
train_df["cosine_sim"] = train_df.apply(cosine_sim, axis = 1)
# train_df["pred_idx_cos"] = train_df["cosine_sim"].apply(lambda x: pred_idx(x))

In [ ]:
train_df

contexts  ... pred_idx_cos
0      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
1      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
2      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            3
3      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
4      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
...                                                  ...  ...          ...
86816  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86817  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86818  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86819  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86820  Kathmandu Metropolitan City (KMC), in order to...  ...            1

[86821 rows x 10 columns]

Save temporary results to drive

In [ ]:
train_df.to_csv(drive_dir+"train_detect_sent.csv", index=None)

### root match (not used)

In [ ]:
predicted = pd.read_csv(drive_dir+"train_detect_sent.csv").reset_index(drop=True)

In [ ]:
doc = en_nlp(predicted.iloc[0,0])

In [ ]:
predicted.iloc[0,1]

In [ ]:
predicted.iloc[0,0]

In [ ]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [ ]:
[to_nltk_tree(sent.root).pretty_print()  for sent in en_nlp(predicted.iloc[0,1]).sents]

In [ ]:
[to_nltk_tree(sent.root) .pretty_print() for sent in doc.sents][1]

In [ ]:
for sent in doc.sents:
    roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]
    print(roots)

In [ ]:
def match_roots(x):
    question = x["question"].lower()
    sentences = en_nlp(x["contexts"].lower()).sents
    
    question_root = st.stem(str([sent.root for sent in en_nlp(question).sents][0]))
    
    li = []
    for i,sent in enumerate(sentences):
        roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]

        if question_root in roots: 
            for k,item in enumerate(ast.literal_eval(x["sentences"])):
                if str(sent) in item.lower(): 
                    li.append(k)
    return li

In [ ]:
predicted["root_match_idx"] = predicted.apply(match_roots, axis = 1)

In [ ]:
predicted["root_match_idx_first"]= predicted["root_match_idx"].apply(lambda x: x[0] if len(x)>0 else 0)

In [ ]:
(predicted["root_match_idx_first"]==predicted["target"]).sum()/predicted.shape[0]

In [ ]:
predicted.to_csv(drive_dir+"train_detect_sent_root.csv", index=None)

# Logistic Regression

## Training

Import data from drive

In [ ]:
data = pd.read_csv(drive_dir+"train_detect_sent.csv").reset_index(drop=True)

In [ ]:
data

contexts  ... pred_idx_cos
0      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
1      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
2      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            3
3      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
4      Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...  ...            2
...                                                  ...  ...          ...
86816  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86817  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86818  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86819  Kathmandu Metropolitan City (KMC), in order to...  ...            1
86820  Kathmandu Metropolitan City (KMC), in order to...  ...            1

[86821 rows x 10 columns]

Prepare for training

In [ ]:
data = data[data["sentences"].apply(lambda x: len(ast.literal_eval(x)))<11].reset_index(drop=True)

In [ ]:
def create_features(data):
    train = pd.DataFrame()
    
    for k in range(len(data["cosine_sim"])):
        dis = ast.literal_eval(data["cosine_sim"][k].replace("nan","1"))
        for i in range(len(dis)):
            train.loc[k, "column_cos_"+"%s"%i] = dis[i]
            
    train["target"] = data["target"]
    return train

In [ ]:
train = create_features(data)

In [ ]:
train

column_cos_0  column_cos_1  ...  column_cos_9  target
0          0.282726      0.271703  ...           NaN       1
1          0.240877      0.163877  ...           NaN       1
2          0.195343      0.187896  ...           NaN       3
3          0.262408      0.200482  ...           NaN       1
4          0.261931      0.251026  ...           NaN       1
...             ...           ...  ...           ...     ...
84342      0.170678      0.126426  ...           NaN       1
84343      0.257391      0.238232  ...           NaN       2
84344      0.193775      0.167717  ...           NaN       2
84345      0.186989      0.150448  ...           NaN       1
84346      0.302323      0.268730  ...           NaN       0

[84347 rows x 11 columns]

In [ ]:
train.fillna(value=0.5, inplace=True)

In [ ]:
train

column_cos_0  column_cos_1  ...  column_cos_9  target
0          0.282726      0.271703  ...           0.5       1
1          0.240877      0.163877  ...           0.5       1
2          0.195343      0.187896  ...           0.5       3
3          0.262408      0.200482  ...           0.5       1
4          0.261931      0.251026  ...           0.5       1
...             ...           ...  ...           ...     ...
84342      0.170678      0.126426  ...           0.5       1
84343      0.257391      0.238232  ...           0.5       2
84344      0.193775      0.167717  ...           0.5       2
84345      0.186989      0.150448  ...           0.5       1
84346      0.302323      0.268730  ...           0.5       0

[84347 rows x 11 columns]

In [ ]:
train.apply(max, axis = 0)

column_cos_0    1.000000
column_cos_1    1.070920
column_cos_2    1.000000
column_cos_3    1.036805
column_cos_4    1.000000
column_cos_5    1.000000
column_cos_6    1.083527
column_cos_7    1.000000
column_cos_8    1.000000
column_cos_9    1.000000
target          9.000000
dtype: float64

Scale the data

In [ ]:
scaler = MinMaxScaler()
X = scaler.fit_transform(train.iloc[:,:-1])

In [ ]:
X_train, X_dev_test, y_train, y_dev_test = train_test_split(X, train.iloc[:,-1], train_size=0.7, random_state=1)

In [ ]:
X_dev, X_test, y_dev, y_test = train_test_split(X, train.iloc[:,-1], train_size=0.5, random_state=2)

Fit data to model

In [ ]:
clf = LogisticRegression(multi_class='multinomial', solver='newton-cg')
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

Make predictions

In [ ]:
y_train_pred = clf.predict(X_train)
y_dev_pred = clf.predict(X_dev)
y_test_pred = clf.predict(X_test)

# Evaluation

## F1 and Accuracy

In [ ]:
def macro_f1(y_true, y_pred):
    y_true = list(y_true)
    TP = [0] * 11
    FP = [0] * 11
    FN = [0] * 11

    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            TP[y_true[i] + 1] += 1
        else:
            FP[y_pred[i] + 1] += 1
            FN[y_true[i] + 1] += 1

    precision = [TP[i] / (TP[i] + FP[i]) if (TP[i] + FP[i])!=0 else 0 for i in range(len(TP))]
    recall = [TP[i] / (TP[i] + FN[i]) for i in range(len(TP))]
    f1 = [2 * precision[i] * recall[i] / (precision[i] + recall[i]) if (precision[i] + recall[i])!=0 else 0 for i in range(len(precision))]

    return sum(f1) / 11

In [ ]:
def accuracy(y_true, y_pred):
    y_true = list(y_true)
    acc = [1 if y_true[i]==y_pred[i] else 0 for i in range(len(y_true))]
    return sum(acc) / len(acc)

In [ ]:
print("Multinomial Logistic regression Train f1-score : ", macro_f1(y_train, y_train_pred))
print("Multinomial Logistic regression Dev f1-score : ", macro_f1(y_dev, y_dev_pred))
print("Multinomial Logistic regression Test f1-score : ", macro_f1(y_test, y_test_pred))

Multinomial Logistic regression Train f1-score :  0.36918675357336733
Multinomial Logistic regression Dev f1-score :  0.36240899016932693
Multinomial Logistic regression Test f1-score :  0.3783394828039804


In [ ]:
print("Multinomial Logistic regression Train acc : ", accuracy(y_train, y_train_pred))
print("Multinomial Logistic regression Dev acc : ", accuracy(y_dev, y_dev_pred))
print("Multinomial Logistic regression Test acc : ", accuracy(y_test, y_test_pred))

Multinomial Logistic regression Train acc :  0.46309406862911145
Multinomial Logistic regression Dev acc :  0.4582552818153795
Multinomial Logistic regression Test acc :  0.4638639920330061


## Error Analysis

Classwise error analysis:

In [ ]:
print(f"Classification report for classifier {clf}:\n"
      f"{metrics.classification_report(y_test, y_test_pred)}\n")

Classification report for classifier LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False):
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00        52
           0       0.42      0.53      0.47      4838
           1       0.52      0.45      0.48      3854
           2       0.51      0.46      0.49      3190
           3       0.46      0.43      0.45      2145
           4       0.46      0.43      0.44      1373
           5       0.44      0.44      0.44       755
           6       0.45      0.36      0.40       366
           7       0.34      0.31      0.32       189
           8       0.50      0.32      0.39        82
           9       0.23      0

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Next, we examine the predictons in details.

In [ ]:
y_test

24702    1
55884    5
36079    2
13825    0
14741    0
        ..
629      0
65756    3
32277    3
10439    1
54664    4
Name: target, Length: 16870, dtype: int64

The model predicts that the answer for test data 24702 is in the second sentence, and the answer for 55884 is in the sixth sentence.

In [ ]:
# Context for 24702
data.iloc[24702]['contexts']

"City and Guilds College was founded in 1876 from a meeting of 16 of the City of London's livery companies for the Advancement of Technical Education (CGLI), which aimed to improve the training of craftsmen, technicians, technologists, and engineers. The two main objectives were to create a Central Institution in London and to conduct a system of qualifying examinations in technical subjects. Faced with their continuing inability to find a substantial site, the Companies were eventually persuaded by the Secretary of the Science and Art Department, General Sir John Donnelly (who was also a Royal Engineer) to found their institution on the eighty-seven acre (350,000 m²) site at South Kensington bought by the 1851 Exhibition Commissioners (for GBP 342,500) for 'purposes of art and science' in perpetuity. The latter two colleges were incorporated by Royal Charter into the Imperial College of Science and Technology and the CGLI Central Technical College was renamed the City and Guilds Colle

In [ ]:
# Quesetion for 24702
data.iloc[24702]['question']

'What were the main objectives of establishing the City and Guilds College?'

In [ ]:
# True answer for 24702
data.iloc[24702]['text']

'to create a Central Institution in London and to conduct a system of qualifying examinations in technical subjects'

The second sentence does contain the true answer. Our model succesfully locates the sentence. Now, look at another test data.

In [ ]:
# Context for 55884
data.iloc[55884]['contexts']

"Following the announcement of Nasser's death, Egypt and the Arab world were in a state of shock. Nasser's funeral procession through Cairo on 1 October was attended by at least five million mourners. The 10-kilometer (6.2 mi) procession to his burial site began at the old RCC headquarters with a flyover by MiG-21 jets. His flag-draped coffin was attached to a gun carriage pulled by six horses and led by a column of cavalrymen. All Arab heads of state attended, with the exception of Saudi King Faisal. King Hussein and Arafat cried openly, and Muammar Gaddafi of Libya fainted from emotional distress twice. A few major non-Arab dignitaries were present, including Soviet Premier Alexei Kosygin and French Prime Minister Jacques Chaban-Delmas."

In [ ]:
# Question for 55883
data.iloc[55884]['question']

'What leader twice fainted during the funeral?'

In [ ]:
# True answer for 55884
data.iloc[55884]['text']

'Muammar Gaddafi'

Again, the predicted sentence contains the true answer. Our model is very robust for all kinds of contexts and questions. 